## Challenge 2: Multi-class Image Classification

Table of Contents:
1. Imports
2. Functions
3. Feature Extraction
4. Exploratory Data Analysis (EDA)
5. 

#### 1. Imports

In [9]:
# To hide deprication warnings 
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

# Append helpers module to use the implemented modules from there
import sys
sys.path.append('../helpers/')

from pathlib import Path
from glob import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pickle
import natsort
import cv2
import csv
import random
import pprint

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score, f1_score, multilabel_confusion_matrix, balanced_accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score, roc_curve

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

# imported from helpers module
from feature_selection import FeatureSelection
from preprocessing import Preprocessing
from feature_extraction import FeatureExtraction

preprocessor = Preprocessing()
feature_extractor = FeatureExtraction()
select_feature = FeatureSelection()

# PPrint object
pp = pprint.PrettyPrinter(indent=4)

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 2. Functions

In [7]:
def pprint_objects(*arg):
    '''Prints large and indented objects clearly.'''
    pp.pprint(arg)

def load_pickle(dir):
    try:
        with open(dir, 'rb') as file:
            return pickle.load(file)
    except:
        raise Exception(f'Error loading the pickle from {dir} directory.')


#### 3. Feature Extraction

In [10]:
# Feature Extraction Code

#### 3. Exploratory Data Analysis